In [16]:
from hotels50k import Hotels50kDataset, UseOriginalTestSplitManager
from powerful_benchmarker.split_managers import IndexSplitManager
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
from tqdm import tqdm

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
dataset = Hotels50kDataset(root='/data/thesis/Hotels-50K', target='hotels', download=True)

Cloning https://github.com/GWUvision/Hotels-50K into /data/thesis/Hotels-50K
Repo already cloned
Unpacking dataset archive
Archive already unpacked
Train images already loaded
Archive already downloaded
Archive already unpacked
Creating symlinks
Creating LMDB image folders
LMDB file already exists
LMDB file already exists
LMDB file already exists
LMDB file already exists
Done downloading and setting up the dataset.
Loading image folders
Concatting dataset
Getting labels


In [19]:
dataset.labels.shape

(2330,)

In [20]:
len(dataset.original_train_dataset.targets), len(dataset.original_test_dataset.targets)

(2274, 56)

In [21]:
import numpy as np

In [22]:
np.where(dataset.labels==1)

(array([   7,    8,    9,   10,   11,   12,   13,   14,   15,   16,   17,
          18,   19,   20,   21,   22,   23,   24,   25,   26,   27,   28,
          29,   30,   31,   32,   33,   34,   35,   36,   37,   38,   39,
        2278, 2279, 2280]),)

In [23]:
assert list(dataset.labels[0:100]) == [dataset[i][1] for i in range(0, 100)]

KeyError: 1

In [24]:
ds = dataset

# first level of dictionary is transform type
# second level consists of separate dataset objects for each split
datasets = {"eval": {"train": ds,
                    "val": ds,
                    "test": ds}
            }

helper_split_manager = UseOriginalTestSplitManager()
split_manager = IndexSplitManager(num_training_partitions=4, num_training_sets=4, helper_split_manager=helper_split_manager)

split_manager.create_split_schemes(datasets)

test_set = split_manager.get_dataset("eval", "test")

for name in split_manager.split_scheme_names:
    split_manager.set_curr_split_scheme(name)
    print("CURR SPLIT SCHEME =", split_manager.curr_split_scheme_name)
    print("train len", len(split_manager.get_dataset("eval", "train")))
    print("val len", len(split_manager.get_dataset("eval", "val")))
    print("test len", len(split_manager.get_dataset("eval", "test")))

    print("train labels", split_manager.get_label_set("eval", "train"))
    print("val labels", split_manager.get_label_set("eval", "val"))
    print("test labels", split_manager.get_label_set("eval", "test"))
    
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "val").indices) == set()
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()
    assert set(split_manager.get_dataset("eval", "val").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()


CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_0
train len 1705
val len 569
test len 56
train labels {19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83}
val labels {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
test labels {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_1
train len 1705
val len 569
test len 56
train labels {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83}
val labels {19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45}
test labels {0, 1, 2, 3, 4, 5, 6, 7, 8,

In [25]:
train_ds = split_manager.get_dataset("eval", "train")
batch_size = 32

def collate(stuff):
    return [s['data'] for s in stuff]

loader = DataLoader(train_ds, shuffle=True, batch_size=batch_size,  num_workers=4, collate_fn=collate)

for batch in tqdm(loader):
    assert len(batch) > 0

100%|██████████| 54/54 [00:03<00:00, 15.54it/s]
